In [ ]:
from sklearn.pipeline import *
from sklearn.naive_bayes import *
from sklearn.cluster import *
from sklearn.covariance import *
from sklearn.cross_decomposition import *
from sklearn.datasets import *
from sklearn.decomposition import *
from sklearn.ensemble import *
from sklearn.feature_extraction import *
from sklearn.feature_extraction.text import *
from sklearn.feature_selection import *
from sklearn.gaussian_process import *
from sklearn.linear_model import *
from sklearn.manifold import *
from sklearn.metrics import *
from sklearn.mixture import *
from sklearn.model_selection import *
from sklearn.neighbors import *
from sklearn.neural_network import *
from sklearn.preprocessing import *
from sklearn_pandas import DataFrameMapper
from sklearn.svm import *
from sklearn.tree import *
from yellowbrick.classifier import ConfusionMatrix, ROCAUC

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from tpot import TPOTClassifier

In [ ]:
titanic = pd.read_csv('train.csv')

In [ ]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
titanic.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
titanic.shape

(891, 12)

In [ ]:
titanic.columns = titanic.columns.str.strip()

In [ ]:
round(titanic['Age'].mean(),0)
titanic['Age'] = titanic['Age'].fillna(round(titanic['Age'].mean(),0))

In [ ]:
titanic['Cabin'].value_counts(),titanic['Cabin'].nunique()

(Cabin
 G6             4
 C23 C25 C27    4
 B96 B98        4
 F2             3
 D              3
               ..
 E17            1
 A24            1
 C50            1
 B42            1
 C148           1
 Name: count, Length: 147, dtype: int64,
 147)

In [ ]:
# Loading data into X-->input and y-->output
# Skipping Name and Ticket because they are unique and won't help with prediction
# Ticket would create too many new columns if we use one-hot encoding
# Dropping Cabin because we got the Deck info from it.

X_raw = titanic.loc[:,['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
y = titanic['Pclass']
X_raw.columns

Index(['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked'], dtype='object')

In [ ]:
#Preprocessing - OneHotEncoding for the job_desig and job_type columns,by dropping originals, and add encoded columns to X_raw
oe = OneHotEncoder(sparse_output = False)
encoded_columns = oe.fit_transform(X_raw[['Sex','Embarked']])
encoded_df = pd.DataFrame(encoded_columns, columns=oe.get_feature_names_out(['Sex','Embarked']))
encoded_df.index = X_raw.index
X_raw = X_raw.drop(columns = ['Sex','Embarked'])
X_raw = pd.concat([X_raw, encoded_df], axis=1)

In [ ]:
scaler = StandardScaler()
X_raw[['Pclass', 'Age','SibSp','Parch','Fare']] = scaler.fit_transform(X_raw[['Pclass', 'Age','SibSp','Parch','Fare']])

In [ ]:
#train-test-split
X_train,X_test,y_train,y_test = train_test_split(X_raw,y,test_size=0.1,random_state=0)

In [ ]:
tpot = TPOTClassifier(generations=5, population_size=50)
tpot

TPOTClassifier()

In [ ]:
tpot.fit(X_train,y_train)

In [ ]:
tpot.predict(X_train)